### Load libraries

In [1]:
import pandas as pd
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from sklearn.metrics.pairwise import cosine_similarity

### Import dataset

In [2]:
df = pd.read_csv("male_players (legacy).csv")

C:\Users\oyund\AppData\Local\Temp\ipykernel_22224\2569618666.py:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("male_players (legacy).csv")


# OPTIMIZATION: Football Player Selection & Acquisition

## Data Pre-Processing

1. **Create Dummies for Roles/Positions**
   - Assign dummy variables for each player's role or position in the team.

&nbsp;

2. **Calculate Adaptation Score for Players**
   - **Factors:**
     1. Whether they have been coached by Pep Guardiola.
     2. Whether they have been in teams coached by coaches similar to Pep Guardiola.

&nbsp;

3. **Create Column for Position Count**
   - Since position column consists of positions separated by a comma, the aim here is to extract those and count the #
   - This could be useful for formulating a constraint based on positional fluidity
   - not necessary

&nbsp;

4. **Separate Work Rate into Two Columns**
   - Columns: `attacking_work_rate` and `defending_work_rate`.
   - Ranking System:
     - Low - 1
     - Medium - 2
     - High - 3

&nbsp;

5. **Identify Injury-Prone Players** (Did not end up using)
   - Analyze `player_traits` to find players prone to injuries.
   - Consider minimizing the presence of such players in one of the objective functions.
   - potentially accounted into other factors, not necessary

&nbsp;
   
5. **Identify Players with Expiring Contracts**
   - Assuming they are more likely to be available for buy out. 


## STEP 1A. DUMMIFY ROLES

In [3]:
############ STEP 1A. DUMMIFY ROLES #################################
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# fill nan values
#df['player_positions'] = df['player_positions'].fillna('')

df['positions_list'] = df['player_positions'].str.split(',')

# strip whitespace and convert to upper case
df['positions_list'] = df['positions_list'].apply(lambda x: [pos.strip().upper() for pos in x])

# apply MultiLabelBinarizer
mlb = MultiLabelBinarizer()
roles_encoded = mlb.fit_transform(df['positions_list'])

roles_df = pd.DataFrame(roles_encoded, columns=mlb.classes_)

df.reset_index(drop=True, inplace=True)
roles_df.reset_index(drop=True, inplace=True)

# Rename columns in roles_df to avoid overlap
roles_df.columns = ['role_' + col for col in roles_df.columns]

# Concatenate
df = pd.concat([df, roles_df], axis=1)

all_columns = df.columns
all_columns_list = all_columns.tolist()
print(all_columns_list)



['player_id', 'player_url', 'fifa_version', 'fifa_update', 'fifa_update_date', 'short_name', 'long_name', 'player_positions', 'overall', 'potential', 'value_eur', 'wage_eur', 'age', 'dob', 'height_cm', 'weight_kg', 'league_id', 'league_name', 'league_level', 'club_team_id', 'club_name', 'club_position', 'club_jersey_number', 'club_loaned_from', 'club_joined_date', 'club_contract_valid_until_year', 'nationality_id', 'nationality_name', 'nation_team_id', 'nation_position', 'nation_jersey_number', 'preferred_foot', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate', 'body_type', 'real_face', 'release_clause_eur', 'player_tags', 'player_traits', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_sp

In [4]:
# Unnecessary step but keeping it for now since other codes rely on this

role_to_formation = {
    'role_GK': 'GK',
    'role_RB': 'RB',
    'role_CB': 'CB',
    'role_LB': 'LB',
    'role_RW': 'RW',
    'role_CM': 'CM',
    'role_LW': 'LW',
    'role_CAM': 'CAM',
    'role_CF': 'CF',
    'role_ST': 'ST',
    'role_CDM': 'CDM', 
    'role_LM': 'LM',   
    'role_RM': 'RM',  
    'role_RWB': 'RWB', 
    'role_LWB': 'LWB',  
}

# Initialize new columns for each formation position
for formation in set(role_to_formation.values()):
    df[formation] = 0

# Update the new columns based on the original role columns
for role, formation in role_to_formation.items():
    if role in df.columns:
        df[formation] += df[role]

# Optionally, drop the original role columns
df.drop(columns=[role for role in role_to_formation.keys()], inplace=True)

#df

## STEP 1B. ADAPTATION SCORE

In [5]:
########## STEP 1B ADAPTATION SCORE #########################

# Filter for the 2016-2017 season
df['fifa_update_date'] = pd.to_datetime(df['fifa_update_date'])
season_start = pd.to_datetime('2015-09-21')
#season_end = pd.to_datetime('2017-06-30')

# Convert the fifa_update_date column to datetime
df['fifa_update_date'] = pd.to_datetime(df['fifa_update_date'])
season_start = pd.to_datetime('2015-09-21')

# List of club_team_ids for teams with similar styles as explained above 
club_team_ids = [21, 241, 22, 9, 18, 112172, 191, 245, 19, 110831, 472, 110556, 39, 448, 1909, 23, 72]

# Filter for player history before the season start and with specified club_team_ids
historical_df = df[(df['fifa_update_date'] < season_start) & (df['club_team_id'].isin(club_team_ids))]

# Check if players were in the specified teams before 2016
player_experience = historical_df.groupby('player_id').size().reset_index(name='counts')

# Merge 
df = df.merge(player_experience[['player_id', 'counts']], on='player_id', how='left')

# Assign adaptation scores 
df['adaptation_score'] = df['counts'].apply(lambda x: 1.0 if x > 0 else 0.5)

df.drop(columns='counts', inplace=True)

## STEP 1C: COLUMN FOR POSITION COUNT

In [6]:
df['position_count'] = df['player_positions'].apply(lambda x: len(x.split(',')))

## STEP 1D: SEPARATE AND RANK WORK_RATE COLUMN

In [7]:
# Split the 'work_rate' column into two new columns
df[['attacking_work_rate', 'defending_work_rate']] = df['work_rate'].str.split('/', expand=True)

def replace_work_rate(value):
    mapping = {'Low': 1, 'Medium': 2, 'High': 3}
    return mapping.get(value.strip().capitalize(), value)

df['attacking_work_rate'] = df['attacking_work_rate'].apply(replace_work_rate)
df['defending_work_rate'] = df['defending_work_rate'].apply(replace_work_rate)

## STEP 1E. INJURY PRONE PLAYERS

In [8]:
df['player_traits'] = df['player_traits'].fillna('')
df['injury_prone'] = df['player_traits'].str.contains('injury prone', case=False).astype(int)

## STEP 1F: CONTRACT EXPIRING SOON

In [9]:
df['club_expiry_soon'] = df['club_contract_valid_until_year'].apply(lambda x: 1 if x in [2016, 2017, 2018, 2019] else 0)

# NOW, OUR MAIN FOCUS: 

### Filter by season: Our choice is 2016-2017. 

In [10]:
#filtered_df = df[(df['fifa_update_date'] >= season_start) & (df['fifa_update_date'] <= season_end)]

filtered_df = df[(df['fifa_update_date'] == season_start)]
#filtered_df

### Our team of choice: Manchester City

In [11]:
# Separate Manchester City players from other teams' players
man_city_players = filtered_df[filtered_df['club_name'] == 'Manchester City'].dropna(subset=['overall'])
other_teams_players = filtered_df[filtered_df['club_name'] != 'Manchester City'].dropna(subset=['overall'])

# Columns to drop
columns_to_drop = ['player_url', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk', 'player_face_url']
other_teams_players.drop(columns=columns_to_drop, inplace=True, errors='ignore')

In [12]:
#man_city_players

#man_city_players.to_csv('man_city.csv', index=False)

In [13]:
#other_teams_players.to_csv("other_teams.csv", index = False)

In [14]:
#Other team players whose attacking_crossing and attacking_finishing is bigger than 70 and 
# player current overall attribute is bigger than 70
other_teams_players = other_teams_players[
    (other_teams_players['attacking_crossing'] > 70) & 
    (other_teams_players['attacking_finishing'] > 70) &
    (other_teams_players['overall'] > 70)
]

# Optimization 1: Selecting 25 players for registration

1. **Initialize Model**
   - This model will focus on maximizing 'potential' score of players in MC. 

&nbsp;

2. **Decision Variables**
     1. Binary variables for Manchester City players.
     2. Binary variables for players from other teams.

&nbsp;

3. **Objective Function**
   - Maximize the 'potential' score of Manchester City based on selected players.
   - Give a boost for players from other teams with higher adaptation scores. 

&nbsp;

4. **Constraints**
   - Define position groups.
   - Budget Constraint: For players being bought, cannot spend no more than 150M euros.
   - Players bought are more likely to be players with expiring contracts.
   - Can buy at most 4 players from other teams
   - The total number of players for registration should be 25. 


In [15]:
import gurobipy as gb
from gurobipy import GRB, quicksum

model = gb.Model("Maximize Team Performance")

# Define decision variables: 1 if a player is chosen for the team, 0 otherwise
player_vars = model.addVars(man_city_players.index, vtype=GRB.BINARY, name="player")
other_player_vars = model.addVars(other_teams_players.index, vtype=GRB.BINARY, name="other_player")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-12


In [16]:
objective_expr = gp.LinExpr()
for i in man_city_players.index:
    # Boost the player's overall rating using the adaptation score
    boosted_overall_rating = man_city_players['potential'][i]
    objective_expr += player_vars[i] * boosted_overall_rating
for i in other_teams_players.index:
    # Boost the player's overall rating using the adaptation score
    boosted_overall_rating = other_teams_players['potential'][i] + (other_teams_players['adaptation_score'][i])
    objective_expr += other_player_vars[i] * boosted_overall_rating
model.setObjective(objective_expr, GRB.MAXIMIZE)

In [17]:
# The budget constraint for trading players should be no more than 150 million Euro
model.addConstr(
    gb.quicksum(other_player_vars[i] * other_teams_players.loc[i, 'value_eur'] for i in other_teams_players.index) <= 150000000,
    "Budget"
)

<gurobi.Constr *Awaiting Model Update*>

In [18]:
# Players with expiring contracts
club_expiry_soon = df['club_expiry_soon']

model.addConstrs(other_player_vars[i] <= club_expiry_soon[i] for i in other_teams_players.index)

{16182: <gurobi.Constr *Awaiting Model Update*>,
 16183: <gurobi.Constr *Awaiting Model Update*>,
 16184: <gurobi.Constr *Awaiting Model Update*>,
 16186: <gurobi.Constr *Awaiting Model Update*>,
 16187: <gurobi.Constr *Awaiting Model Update*>,
 16188: <gurobi.Constr *Awaiting Model Update*>,
 16189: <gurobi.Constr *Awaiting Model Update*>,
 16192: <gurobi.Constr *Awaiting Model Update*>,
 16197: <gurobi.Constr *Awaiting Model Update*>,
 16198: <gurobi.Constr *Awaiting Model Update*>,
 16199: <gurobi.Constr *Awaiting Model Update*>,
 16200: <gurobi.Constr *Awaiting Model Update*>,
 16201: <gurobi.Constr *Awaiting Model Update*>,
 16202: <gurobi.Constr *Awaiting Model Update*>,
 16205: <gurobi.Constr *Awaiting Model Update*>,
 16206: <gurobi.Constr *Awaiting Model Update*>,
 16207: <gurobi.Constr *Awaiting Model Update*>,
 16208: <gurobi.Constr *Awaiting Model Update*>,
 16209: <gurobi.Constr *Awaiting Model Update*>,
 16212: <gurobi.Constr *Awaiting Model Update*>,
 16213: <gurobi.Cons

In [19]:
model.addConstr(quicksum(other_player_vars[i] for i in other_teams_players.index) <= 4, "Other_Teams_Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [20]:
model.addConstr(
    quicksum(player_vars[i] for i in man_city_players.index) + 
    quicksum(other_player_vars[i] for i in other_teams_players.index) == 25
)

<gurobi.Constr *Awaiting Model Update*>

## Position Constraints:

In [21]:
model.addConstr(quicksum(player_vars[i] * man_city_players.loc[i, 'CB'] for i in man_city_players.index) + 
               quicksum(other_player_vars[i] * other_teams_players.loc[i, 'CB'] for i in other_teams_players.index) >=4)

<gurobi.Constr *Awaiting Model Update*>

In [22]:
model.addConstr(quicksum(player_vars[i] * man_city_players.loc[i, 'LB'] for i in man_city_players.index) + 
               quicksum(other_player_vars[i] * other_teams_players.loc[i, 'LB'] for i in other_teams_players.index) >=2)

<gurobi.Constr *Awaiting Model Update*>

In [23]:
model.addConstr(quicksum(player_vars[i] * man_city_players.loc[i, 'RB'] for i in man_city_players.index) + 
               quicksum(other_player_vars[i] * other_teams_players.loc[i, 'RB'] for i in other_teams_players.index) >= 2)

<gurobi.Constr *Awaiting Model Update*>

In [24]:
model.addConstr(quicksum(player_vars[i] * man_city_players.loc[i, 'CM'] for i in man_city_players.index) +
               quicksum(other_player_vars[i] * other_teams_players.loc[i, 'CM'] for i in other_teams_players.index) >= 4)

<gurobi.Constr *Awaiting Model Update*>

In [25]:
model.addConstr(quicksum(player_vars[i] * man_city_players.loc[i, 'ST'] for i in man_city_players.index) + 
               quicksum(other_player_vars[i] * other_teams_players.loc[i, 'ST'] for i in other_teams_players.index) >=1)

<gurobi.Constr *Awaiting Model Update*>

In [26]:
model.addConstr(quicksum(player_vars[i] * man_city_players.loc[i, 'CF'] for i in man_city_players.index) + 
               quicksum(other_player_vars[i] * other_teams_players.loc[i, 'CF'] for i in other_teams_players.index) >= 1)

<gurobi.Constr *Awaiting Model Update*>

In [27]:
model.addConstr(quicksum(player_vars[i] * man_city_players.loc[i, 'RW'] for i in man_city_players.index) + 
               quicksum(other_player_vars[i] * other_teams_players.loc[i, 'RW'] for i in other_teams_players.index) >= 1)

<gurobi.Constr *Awaiting Model Update*>

In [28]:
model.addConstr(quicksum(player_vars[i] * man_city_players.loc[i, 'LW'] for i in man_city_players.index) + 
               quicksum(other_player_vars[i] * other_teams_players.loc[i, 'LW'] for i in other_teams_players.index) >= 1)

<gurobi.Constr *Awaiting Model Update*>

In [29]:
model.addConstr(quicksum(player_vars[i] * man_city_players.loc[i, 'GK'] for i in man_city_players.index) + 
               quicksum(other_player_vars[i] * other_teams_players.loc[i, 'GK'] for i in other_teams_players.index) >= 2)

<gurobi.Constr *Awaiting Model Update*>

In [30]:
model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i5-1335U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 323 rows, 342 columns and 1631 nonzeros
Model fingerprint: 0x507c54b2
Variable types: 0 continuous, 342 integer (342 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+08]
  Objective range  [6e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 2044.0000000
Presolve removed 311 rows and 64 columns
Presolve time: 0.00s
Presolved: 12 rows, 278 columns, 1090 nonzeros
Variable types: 0 continuous, 278 integer (259 binary)
Found heuristic solution: objective 2107.5000000

Root relaxation: objective 2.111408e+03, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |  

In [31]:
model.optimize()
print('Selected Players:')
for i in man_city_players.index:
    if player_vars[i].x > 0.5: 
        print(man_city_players.loc[i, 'long_name'])
for i in other_teams_players.index:
    if other_player_vars[i].x > 0.5:
        print(other_teams_players.loc[i, 'long_name'])


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i5-1335U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 323 rows, 342 columns and 1631 nonzeros
Model fingerprint: 0x507c54b2
Variable types: 0 continuous, 342 integer (342 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+08]
  Objective range  [6e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolved: 12 rows, 278 columns, 1090 nonzeros

Continuing optimization...


Explored 1 nodes (15 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 3: 2110.5 2107.5 2044 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.110500000000e+03, best bound 2.110500000000e+03, gap 0.0000%
Sele

In [32]:
import pandas as pd

selected_players = []

for i in man_city_players.index:
    if player_vars[i].x > 0.5: 
        selected_players.append(man_city_players.loc[i, 'long_name'])
        
for i in other_teams_players.index:
    if other_player_vars[i].x > 0.5:
        selected_players.append(other_teams_players.loc[i, 'long_name'])

# Create a DataFrame from the selected players list
df_selected_players = pd.DataFrame(selected_players, columns=['Player Name'])

# Display the DataFrame
print(df_selected_players)

                          Player Name
0           David Josué Jiménez Silva
1   Sergio Leonel Agüero del Castillo
2                     Kevin De Bruyne
3                 Gnégnéri Yaya Touré
4           Vincent Jean Mpoy Kompany
5            Charles Joseph John Hart
6     Nicolás Hernán Gonzalo Otamendi
7                         Samir Nasri
8         Pablo Javier Zabaleta Girod
9                     Raheem Sterling
10           Martín Gastón Demichelis
11             Wilfried Guemiand Bony
12                Gaël Dimitri Clichy
13                   Eliaquim Mangala
14                       Bacary Sagna
15                       Fabian Delph
16                     Bruno Zuculini
17          Kelechi Promise Iheanacho
18                    Thierry Ambrose
19                         Angus Gunn
20                      David Faupala
21      Neymar da Silva Santos Júnior
22                        Mario Götze
23        Christian Dannemann Eriksen
24     Raphaël Adelino José Guerreiro


In [33]:
import pandas as pd

all_players = pd.concat([man_city_players, other_teams_players])

selected_players_df = all_players[all_players['long_name'].isin(selected_players)]
#selected_players_df.to_csv('man.csv', index=False)

## Make sure to have imported man.csv from the previous step
* the code runs in our directory, contact oyundari.batbayar@mail.mcgill.ca if you encounter issues. 

In [34]:
df = pd.read_csv('man.csv',encoding='latin1')
Id = df['player_id'].tolist()
print(Id)
df_all = pd.read_csv("male_players (legacy).csv",encoding='latin1')
df_all['fifa_update_date'] = pd.to_datetime(df_all['fifa_update_date'])
df_all = df_all[df_all['fifa_update_date'].dt.year == 2016]
Id = df['player_id'].tolist()
df = df_all[df_all['player_id'].isin(Id)]
df['preferred_foot'] = df['preferred_foot'].apply(lambda x: 0 if x == 'Right' else 1)

[168542, 153079, 192985, 20289, 139720, 150724, 192366, 165239, 142784, 202652, 134979, 189963, 152554, 190531, 163419, 183427, 203864, 225719, 221350, 216325, 229670, 190871, 192318, 190460, 209889]


C:\Users\oyund\AppData\Local\Temp\ipykernel_22224\2395556714.py:4: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv("male_players (legacy).csv",encoding='latin1')
C:\Users\oyund\AppData\Local\Temp\ipykernel_22224\2395556714.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preferred_foot'] = df['preferred_foot'].apply(lambda x: 0 if x == 'Right' else 1)


In [35]:
df[['attacking_work_rate', 'defending_work_rate']] = df['work_rate'].str.split('/', expand=True)

def replace_work_rate(value):
    mapping = {'Low': 1, 'Medium': 2, 'High': 3}
    return mapping.get(value.strip().capitalize(), value)

df['attacking_work_rate'] = df['attacking_work_rate'].apply(replace_work_rate)
df['defending_work_rate'] = df['defending_work_rate'].apply(replace_work_rate)

C:\Users\oyund\AppData\Local\Temp\ipykernel_22224\441303172.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['attacking_work_rate', 'defending_work_rate']] = df['work_rate'].str.split('/', expand=True)
C:\Users\oyund\AppData\Local\Temp\ipykernel_22224\441303172.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['attacking_work_rate', 'defending_work_rate']] = df['work_rate'].str.split('/', expand=True)
C:\Users\oyund\AppData\Local\Temp\ipykernel_22224\441303172.py:7: SettingWithCopyWarning: 
A v

In [36]:
name = df["short_name"].tolist()
pn = df.iloc[:, -30:-3].columns
names = [p+' : '+ item for item in name for p in pn]
foot = df['preferred_foot'].tolist()
overall = df["overall"].tolist()
potential = df['potential'].tolist()
attack = df['attacking_finishing'].tolist()
defend_heading = df['defending_marking_awareness']+df['attacking_heading_accuracy']
defend_heading_list = defend_heading.tolist()
speed = df['movement_sprint_speed'].tolist()
cross = df['attacking_crossing'].tolist()
stamina = df["power_stamina"].tolist()
tackle = df['defending_standing_tackle'].tolist()
defawareness = df['defending_marking_awareness'].tolist()
control = df['skill_ball_control'].tolist()
positioning = df['mentality_positioning'].tolist()
defend_work_rate = df['defending_work_rate'].tolist()
strength = df['power_strength'].tolist()

num = len(name)
ability = df.iloc[:, -30:-3].values
for i in range(len(ability)):
    for j in range(len(ability[i])):
        ability[i][j] = eval(ability[i][j])
        if ability[i][j] not in range(0,100):
            print(ability[i][j])

In [37]:
new_26_ability = []
new_26_name1 = []
new_foot = []
new_speed = []
new_cross = []
new_stamina = []
new_tackle = []
new_defawareness = []
new_control = []
new_positioning = []
new_defend_work_rate = []
new_strength = []

for i in range(num):
    new_26_ability.append(ability[i].tolist())
    new_26_name1.append(name[i])
    new_foot.append(foot[i])
    new_speed.append(speed[i])
    new_cross.append(cross[i])
    new_stamina.append(stamina[i])
    new_tackle.append(tackle[i])
    new_defawareness.append(defawareness[i])
    new_control.append(control[i])
    new_positioning.append(positioning[i])
    new_defend_work_rate.append(defend_work_rate[i])
    new_strength.append(strength[i])
    

num = len(new_26_name1)
new_26_name = [p+' : '+ item for item in new_26_name1 for p in pn]

In [38]:
# attack formation 433
model = gb.Model("2016 best11")
#model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Your code here:
n = model.addVars(num,27,vtype = GRB.BINARY,name=new_26_name)
sc = model.addVars(num,vtype = GRB.BINARY,name=new_26_name1)
dn = model.addVars(num,vtype = GRB.BINARY,name=new_26_name1)
stn = model.addVars(num,vtype = GRB.BINARY,name=new_26_name1)

total = quicksum(n[i,j]*new_26_ability[i][j] for i in range(num) for j in range(27))
controling = quicksum(sc[i]*new_control[i] for i in range(num))
staminaing = quicksum(stn[i]*new_stamina[i] for i in range(num))
defending = quicksum(dn[i]*new_defend_work_rate[i] for i in range(num))

model.setObjectiveN(staminaing, index =3, priority = 0)
model.setObjectiveN(defending, index =2, priority = 1)
model.setObjectiveN(controling, index =1, priority = 2)
model.setObjectiveN(total/11, index =0, priority = 3)
model.ModelSense = GRB.MAXIMIZE

model.addConstr(sum(sc[i] for i in range(num))==11)
model.addConstr(sum(dn[i] for i in range(num))==11)
model.addConstr(sum(stn[i] for i in range(num))==11)

for i in range(num):
    model.addConstr(sc[i] <= sum(n[i,j] for j in range(27)))
    model.addConstr(dn[i] <= sum(n[i,j] for j in range(27)))
    model.addConstr(stn[i] <= sum(n[i,j] for j in range(27)))

for i in range(num):
    model.addConstr(sum(n[i,j] for j in range(27))<=1)
model.addConstr(sum(n[i,j] for i in range(num) for j in range(27))==11)

# goal keeper equal to 1
model.addConstr(sum(n[i,26] for i in range(num))==1)

# forward at least 1
model.addConstr(sum(n[i,1] for i in range(num)) + sum(n[i,5] for i in range(num))==1)

# lb,lwb and rb,rwb only 1 each
model.addConstr(sum(n[i,21] for i in range(num)) ==1)

model.addConstr(sum(n[i,25] for i in range(num)) ==1)

# rcb and lcb only 1 each
model.addConstr(sum(n[i,22] for i in range(num)) ==1)
model.addConstr(sum(n[i,24] for i in range(num)) ==1)

# lb must has preferred_foot at left and rb must has preferred_foot at right

model.addConstrs(n[i,21]<= new_foot[i] for i in range(num))
model.addConstrs(n[i,16]<= new_foot[i] for i in range(num))

model.addConstrs(n[i,25] + new_foot[i] <= 1 for i in range(num))
model.addConstrs(n[i,20] + new_foot[i] <= 1 for i in range(num))

# Each position only can be 1 players
model.addConstrs(sum(n[i,j] for i in range(num)) <=1 for j in range(27))

# Number of left side and Right side should be equal
l = sum(n[i,0] +n[i,3]+n[i,4]+n[i,8]+n[i,11]+n[i,12]+n[i,16]+n[i,17]+n[i,21] for i in range(num))
r = sum(n[i,2] +n[i,6]+n[i,7]+n[i,10]+n[i,14]+n[i,15]+n[i,19]+n[i,20]+n[i,25] for i in range(num))
model.addConstr(l==r)

rf = sum(n[i,6]+n[i,7]+n[i,15]+n[i,10] for i in range(num))
lf = sum(n[i,3]+n[i,4]+n[i,11]+n[i,8] for i in range(num))
model.addConstr(lf==rf)

# Midfield exactly 3
f = sum(n[i,9]+ n[i,12] +n[i,13]+n[i,14]+n[i,17]+n[i,18]+n[i,19] for i in range(num))
model.addConstr(f ==3)

# make sure not all locate at center
model.addConstr(sum(n[i,9]+ n[i,13] +n[i,18] for i in range(num)) ==1)


# Two wine
w = sum(n[i,3]+ n[i,7] +n[i,11]+n[i,15]for i in range(num))
model.addConstr(w ==2)

<gurobi.Constr *Awaiting Model Update*>

In [39]:
model.optimize()
for i in range(num):
        for j in range(27):
            if n[i, j].X > 0.5:  # Assuming a binary variable, check if it's set to 1
                print(n[i, j].VarName)
print('overall rating is: ',model.getObjective(0).getValue())

rw : Neymar
st : S. Agüero
lm : K. De Bruyne
cam : David Silva
rcb : V. Kompany
lcb : N. Otamendi
rcm : Y. Touré
gk : J. Hart
lcm : C. Eriksen
rb : P. Zabaleta
lb : G. Clichy
overall rating is:  85.36363636363637


In [40]:
# attack formation 4231
model = gb.Model("2021 best11")
#model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Your code here:
n = model.addVars(num,27,vtype = GRB.BINARY,name=new_26_name)
sc = model.addVars(num,vtype = GRB.BINARY,name=new_26_name1)
dn = model.addVars(num,vtype = GRB.BINARY,name=new_26_name1)
strength = model.addVars(num,vtype = GRB.BINARY,name=new_26_name1)

total = quicksum(n[i,j]*new_26_ability[i][j] for i in range(num) for j in range(27))
controling = quicksum(sc[i]*new_control[i] for i in range(num))
strengthing = quicksum(strength[i]*new_strength[i] for i in range(num))
defending = quicksum(dn[i]*new_defend_work_rate[i] for i in range(num))

model.setObjectiveN(strengthing, index =3, priority = 0)
model.setObjectiveN(defending, index =2, priority = 1)
model.setObjectiveN(controling, index =1, priority = 2)
model.setObjectiveN(total/11, index =0, priority = 3)
model.ModelSense = GRB.MAXIMIZE

model.addConstr(sum(sc[i] for i in range(num))==11)
model.addConstr(sum(dn[i] for i in range(num))==11)
model.addConstr(sum(strength[i] for i in range(num))==11)

for i in range(num):
    model.addConstr(sc[i] <= sum(n[i,j] for j in range(27)))
    model.addConstr(dn[i] <= sum(n[i,j] for j in range(27)))
    model.addConstr(strength[i] <= sum(n[i,j] for j in range(27)))

for i in range(num):
    model.addConstr(sum(n[i,j] for j in range(27))<=1)
model.addConstr(sum(n[i,j] for i in range(num) for j in range(27))==11)

# goal keeper equal to 1
model.addConstr(sum(n[i,26] for i in range(num))==1)

# forward at least 1
model.addConstr(sum(n[i,1] for i in range(num)) + sum(n[i,5] for i in range(num))==1)

# cam equal to 1
model.addConstr(sum(n[i,9] for i in range(num))==1)

# dm at least 1 and no more two
model.addConstr(sum(n[i,16] for i in range(num)) + sum(n[i,17] for i in range(num))+ sum(n[i,18] for i in range(num))>=1)
model.addConstr(sum(n[i,16] for i in range(num)) + sum(n[i,17] for i in range(num))+ sum(n[i,18] for i in range(num)) <=2)

# lb,lwb and rb,rwb only 1 each
model.addConstr(sum(n[i,21] for i in range(num)) ==1)

model.addConstr(sum(n[i,25] for i in range(num)) ==1)

# rcb and lcb only 1 each
model.addConstr(sum(n[i,22] for i in range(num)) ==1)
model.addConstr(sum(n[i,24] for i in range(num)) ==1)

# lb must has preferred_foot at left and rb must has preferred_foot at right

model.addConstrs(n[i,21]<= new_foot[i] for i in range(num))
model.addConstrs(n[i,16]<= new_foot[i] for i in range(num))

model.addConstrs(n[i,25] + new_foot[i] <= 1 for i in range(num))
model.addConstrs(n[i,20] + new_foot[i] <= 1 for i in range(num))

# Each position only can be 1 players
model.addConstrs(sum(n[i,j] for i in range(num)) <=1 for j in range(27))

# Number of left side and Right side should be equal
l = sum(n[i,0] +n[i,3]+n[i,4]+n[i,8]+n[i,11]+n[i,12]+n[i,16]+n[i,17]+n[i,21] for i in range(num))
r = sum(n[i,2] +n[i,6]+n[i,7]+n[i,10]+n[i,14]+n[i,15]+n[i,19]+n[i,20]+n[i,25] for i in range(num))
model.addConstr(l==r)

rf = sum(n[i,6]+n[i,7]+n[i,15]+n[i,10] for i in range(num))
lf = sum(n[i,3]+n[i,4]+n[i,11]+n[i,8] for i in range(num))
model.addConstr(lf==rf)

# Midfield exactly 3
f = sum(n[i,9]+ n[i,12] +n[i,13]+n[i,14]+n[i,17]+n[i,18]+n[i,19] for i in range(num))
model.addConstr(f ==3)

# make sure not all locate at center
model.addConstr(sum(n[i,9]+ n[i,13] +n[i,18] for i in range(num)) ==1)


# Two wine
w = sum(n[i,3]+ n[i,7] +n[i,11]+n[i,15]for i in range(num))
model.addConstr(w ==2)

<gurobi.Constr *Awaiting Model Update*>

In [41]:
model.optimize()
for i in range(num):
        for j in range(27):
            if n[i, j].X > 0.5:  # Assuming a binary variable, check if it's set to 1
                print(n[i, j].VarName)
print('overall rating is: ',model.getObjective(0).getValue())

lw : Neymar
st : S. Agüero
rm : K. De Bruyne
cam : David Silva
lcb : V. Kompany
rcb : N. Otamendi
rcm : Y. Touré
gk : J. Hart
ldm : P. Zabaleta
rb : B. Sagna
lb : G. Clichy
overall rating is:  85.36363636363637
